# 📱 Pillow Instagram Templates (v1)
Fixed-size **540 × 675** templates ready for Instagram posts. Each example has a background (solid color, gradient, or URL image) and clear text (title + description). Includes a **three-events** layout and **bottom overlay** for readability over photos.

**How to use**
1) Run the setup cell.
2) Pick a template, tweak the parameters (title, description, colors, image URL), and run.
3) Files are saved to `OUTPUT_DIR`.


## 0) Setup

In [ ]:
#@title Install & Imports { display-mode: "form" }
!pip -q install pillow requests rich

from PIL import Image, ImageOps, ImageEnhance, ImageFilter, ImageDraw, ImageFont, ImageColor
import requests, io, os, textwrap, math
from IPython.display import display
from rich import print

W, H = 540, 675
OUTPUT_DIR = "/content/IG_Templates"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEFAULT_FONT = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
def get_font(size=40):
    try:
        return ImageFont.truetype(DEFAULT_FONT, size=size)
    except Exception:
        return ImageFont.load_default()

def wrap_text(txt, font, max_width):
    # Wrap text to fit within max_width using textbbox
    if not txt:
        return ""
    words = txt.split()
    lines, line = [], ""
    dummy_img = Image.new("RGB", (W, H))
    d = ImageDraw.Draw(dummy_img)
    for w_ in words:
        test = (w_ if not line else line + " " + w_)
        bbox = d.textbbox((0,0), test, font=font)
        if bbox[2] <= max_width:
            line = test
        else:
            lines.append(line)
            line = w_
    if line:
        lines.append(line)
    return "\n".join(lines)

def fetch_image(url, size=(W, H), crop=True):
    # Download an image and fit to canvas
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    im = Image.open(io.BytesIO(r.content)).convert("RGB")
    if crop:
        im = ImageOps.fit(im, size, Image.LANCZOS)
    else:
        im = im.resize(size, Image.LANCZOS)
    return im

print(f"[green]Ready.[/green] Canvas: {W}×{H}  |  Output: {OUTPUT_DIR}")

## 1) Template: Solid Color Background + Title + Description

In [ ]:
def solid_bg_card(
    title="Workshop: Image Magic",
    description="Hands-on intro to Pillow (PIL). Learn drawing, text, filters, and templates — in minutes.",
    bg_color="#0b1220",
    title_color="#ffffff",
    desc_color="#cbd5e1",
    out_name="solid_bg_card.png",
):
    img = Image.new("RGB", (W, H), bg_color)
    d = ImageDraw.Draw(img)
    title_font = get_font(54)   # sized for 540x675
    desc_font  = get_font(28)

    # Margins
    pad_x = 38
    pad_y = 44

    # Title
    title_wrap = wrap_text(title, title_font, W - 2*pad_x)
    d.multiline_text((pad_x, pad_y), title_wrap, fill=title_color, font=title_font, spacing=6)

    # Description under title
    bbox = d.multiline_textbbox((pad_x, pad_y), title_wrap, font=title_font, spacing=6)
    desc_top = bbox[3] + 18
    desc_wrap = wrap_text(description, desc_font, W - 2*pad_x)
    d.multiline_text((pad_x, desc_top), desc_wrap, fill=desc_color, font=desc_font, spacing=6)

    out_path = f"{OUTPUT_DIR}/{out_name}"
    img.save(out_path, quality=95)
    display(img); print("Saved:", out_path)

solid_bg_card()

## 2) Template: Gradient Background + Title + Description

In [ ]:
def gradient_bg_card(
    title="New Episode: Creative Coding",
    description="We explore text-on-image layouts, gradients, and readability tricks for social posts.",
    grad_dark="#0ea5e9",
    grad_light="#a78bfa",
    title_color="#0b1220",
    desc_color="#1f2937",
    out_name="gradient_bg_card.png",
):
    # Make vertical gradient
    g = Image.linear_gradient("L").resize((W, H))
    g = ImageOps.colorize(g, black=grad_dark, white=grad_light)
    img = g.convert("RGB")
    d = ImageDraw.Draw(img)

    title_font = get_font(54)
    desc_font  = get_font(28)
    pad_x, pad_y = 34, 42

    title_wrap = wrap_text(title, title_font, W - 2*pad_x)
    d.multiline_text((pad_x, pad_y), title_wrap, fill=title_color, font=title_font, spacing=6)

    bbox = d.multiline_textbbox((pad_x, pad_y), title_wrap, font=title_font, spacing=6)
    desc_top = bbox[3] + 18
    desc_wrap = wrap_text(description, desc_font, W - 2*pad_x)
    # Add subtle translucent white panel behind paragraph
    panel = Image.new("RGBA", (W - 2*pad_x, d.multiline_textbbox((0,0), desc_wrap, font=desc_font, spacing=6)[3]), (255,255,255,100))
    img_rgba = img.convert("RGBA")
    img_rgba.paste(panel, (pad_x, desc_top-8), panel)
    d = ImageDraw.Draw(img_rgba)
    d.multiline_text((pad_x, desc_top), desc_wrap, fill=desc_color, font=desc_font, spacing=6)

    out_path = f"{OUTPUT_DIR}/{out_name}"
    img_rgba.convert("RGB").save(out_path, quality=95)
    display(img_rgba); print("Saved:", out_path)

gradient_bg_card()

## 3) Template: Three Events Listed (Date • Title • Details)

In [ ]:
def three_events_card(
    header="Upcoming Events",
    events=(
        ("Sep 12", "Intro to Pillow", "Basics of drawing, text, and filters"),
        ("Sep 18", "FFmpeg in Colab", "Video filters, CRF, GIFs"),
        ("Sep 25", "Design Hacks", "Readable layouts for social posts"),
    ),
    bg_color="#111827",
    header_color="#22d3ee",
    text_color="#e5e7eb",
    muted="#94a3b8",
    out_name="three_events_card.png",
):
    img = Image.new("RGB", (W, H), bg_color)
    d = ImageDraw.Draw(img)
    header_font = get_font(50)
    date_font   = get_font(28)
    title_font  = get_font(32)
    body_font   = get_font(24)

    pad_x, pad_y = 32, 36

    # Header
    d.text((pad_x, pad_y), header, fill=header_color, font=header_font)
    y = d.textbbox((pad_x, pad_y), header, font=header_font)[3] + 16

    # Divider
    d.line((pad_x, y, W - pad_x, y), fill="#1f2937", width=3)
    y += 16

    # Event rows
    row_gap = 18
    for date, title, details in events:
        # Date pill
        date_text = date.upper()
        dw, dh = d.textbbox((0,0), date_text, font=date_font)[2:]
        pad = 8
        pill_w = dw + 2*pad
        pill_h = dh + 2*pad
        d.rounded_rectangle((pad_x, y, pad_x+pill_w, y+pill_h), radius=10, fill="#0b1220")
        d.text((pad_x+pad, y+pad), date_text, fill="#a5b4fc", font=date_font)

        # Title + details to the right
        x_text = pad_x + pill_w + 14
        title_wrap = wrap_text(title, title_font, W - x_text - pad_x)
        d.multiline_text((x_text, y), title_wrap, fill=text_color, font=title_font)

        tb = d.multiline_textbbox((x_text, y), title_wrap, font=title_font)
        y_details = tb[3] + 4
        details_wrap = wrap_text(details, body_font, W - x_text - pad_x)
        d.multiline_text((x_text, y_details), details_wrap, fill=muted, font=body_font)
        y = d.multiline_textbbox((x_text, y_details), details_wrap, font=body_font)[3] + row_gap

    out_path = f"{OUTPUT_DIR}/{out_name}"
    img.save(out_path, quality=95)
    display(img); print("Saved:", out_path)

three_events_card()

## 4) Template: Image Background (URL) + Bottom Overlay + Title/Description

In [ ]:
def photo_bottom_overlay_card(
    image_url="https://images.unsplash.com/photo-1522202176988-66273c2fd55f?q=80&w=1200&auto=format&fit=crop",
    title="Creative Jam 2025",
    description="Hands-on design & code mini-workshops. Bring your laptop and curiosity.",
    overlay_color=(0,0,0,180),  # semi-transparent black
    title_color="#ffffff",
    desc_color="#e5e7eb",
    out_name="photo_bottom_overlay_card.png",
):
    # Background photo
    bg = fetch_image(image_url, size=(W, H), crop=True).convert("RGBA")

    # Bottom overlay panel
    d = ImageDraw.Draw(bg)
    panel_h = int(H*0.38)
    panel = Image.new("RGBA", (W, panel_h), overlay_color)
    bg.paste(panel, (0, H-panel_h), panel)

    # Text on top of overlay
    title_font = get_font(46)
    desc_font  = get_font(26)
    pad_x = 28
    text_w = W - 2*pad_x

    title_wrap = wrap_text(title, title_font, text_w)
    desc_wrap  = wrap_text(description, desc_font, text_w)

    # place text near bottom with spacing
    y = H - panel_h + 18
    d.multiline_text((pad_x, y), title_wrap, fill=title_color, font=title_font, spacing=4)
    tb = d.multiline_textbbox((pad_x, y), title_wrap, font=title_font, spacing=4)
    y = tb[3] + 10
    d.multiline_text((pad_x, y), desc_wrap, fill=desc_color, font=desc_font, spacing=4)

    out_path = f"{OUTPUT_DIR}/{out_name}"
    bg.convert("RGB").save(out_path, quality=95)
    display(bg); print("Saved:", out_path)

photo_bottom_overlay_card()

## 5) Template: Photo with Center Badge + Title/Description

In [ ]:
def photo_center_badge_card(
    image_url="https://images.unsplash.com/photo-1475724017904-b712052c192a?q=80&w=1200&auto=format&fit=crop",
    title="Studio Session",
    description="Portrait lighting, color grading, and mood boards.",
    badge_text="FREE ENTRY",
    out_name="photo_center_badge_card.png",
):
    bg = fetch_image(image_url, size=(W, H)).convert("RGBA")

    # Soft gradient overlay from top for text readability
    grad = Image.linear_gradient("L").resize((W, H))
    grad = ImageOps.colorize(grad, black=(0,0,0,60), white=(0,0,0,0)).convert("RGBA")
    bg = Image.alpha_composite(bg, grad)

    d = ImageDraw.Draw(bg)
    title_font = get_font(48)
    desc_font  = get_font(26)
    badge_font = get_font(22)

    # Badge in center
    bw, bh = d.textbbox((0,0), badge_text, font=badge_font)[2:]
    pad = 14
    bx = (W - (bw + 2*pad)) // 2
    by = int(H*0.40)
    d.rounded_rectangle((bx, by, bx+bw+2*pad, by+bh+2*pad), radius=12, fill="#22d3ee")
    d.text((bx+pad, by+pad), badge_text, fill="#0b1220", font=badge_font)

    # Title / Description lower
    pad_x = 28
    text_w = W - 2*pad_x
    y = by + bh + 2*pad + 20

    t_wrap = wrap_text(title, title_font, text_w)
    d.multiline_text((pad_x, y), t_wrap, fill="#ffffff", font=title_font, spacing=4)
    tb = d.multiline_textbbox((pad_x, y), t_wrap, font=title_font, spacing=4)
    y = tb[3] + 8

    d_wrap = wrap_text(description, desc_font, text_w)
    d.multiline_text((pad_x, y), d_wrap, fill="#e5e7eb", font=desc_font, spacing=4)

    out_path = f"{OUTPUT_DIR}/{out_name}"
    bg.convert("RGB").save(out_path, quality=95)
    display(bg); print("Saved:", out_path)

photo_center_badge_card()

---
### Notes
- All canvases are fixed to **540×675** for Instagram portrait posts.
- Change fonts by pointing `DEFAULT_FONT` to a TTF you upload (or mount Google Drive).
- The photo templates fetch images from URLs; replace with your own links.
- For persistence, save to Drive (mount at `/content/drive`) and change `OUTPUT_DIR`.
